## Day 20 - Modules & Pulses

**Part 1: Multiply the total amount of low beam by the total amount of high beam after 1000 button presses**

Start with low pulse

Modules:
- `broadcaster` just sends one to many as specified
- `%` flip-flop modules. Either *on* or *off* (initially off). If flip flop recieves HIGH, do nothing. If flip flop recieves LOW:
  - If it was **off**, it turns *on* and sends a **high** pulse. If it was **on** it turns *off* and sends a **low** pulse.
- `&` Conjunction modules. Remember the type of the most recent pulse recieved from *each* of their connected input modules. Initially default ot remembering a *low pulse* for each.
  - When a pulse is recieved, conjunction updates its memory for *THAT* input
    - Then if it remembers *high pulses* for all inputs, it sends a *low pulse*, otherwise it sends a *high pulse*.

When press button, you send a single *low pulse* directly to the `broadcaster` module.
After pressing you wait until all pulses delievered and fully handled before pushing it again.
!! Pulses are always processed *in the order they are sent*.

In [2]:
with open("./example1.txt") as f:
    example1_lines = [line.strip() for line in f.readlines()]

with open("./example2.txt") as f:
    example2_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example1_lines

['broadcaster -> a, b, c', '%a -> b', '%b -> c', '%c -> inv', '&inv -> a']

In [3]:
def get_mappings(lines: list[str]) -> dict:
    mappings = {}

    for line in lines:
        src, dest = line.split(" -> ")
        dest = dest.split(", ")

        if src == "broadcaster":
            mappings[src] = dest
        elif src[0] == "%":
            mappings[src[1:]] = (src[0], dest, False)  # % type [...] destination(s) False state (off!)
        elif src[0] == "&":
            mappings[src[1:]] = (src[0], dest, {})
        else:
            raise ValueError("what kind of prefix is this?", src[0])

    nodes = [k for k in mappings.keys() if k != "broadcaster"]
    for k, v in mappings.items():
        if k == "broadcaster":
            continue
        type_, dest, _ = v
        if type_ == "&":
            connected_inputs = []
            for node in nodes:
                _, tmp_dest, _ = mappings[node]
                if k in tmp_dest:
                    connected_inputs.append(node)
            states = {node: False for node in connected_inputs}
            mappings[k] = (type_, dest, states)
    
    return mappings

In [4]:
from dataclasses import dataclass

@dataclass
class Pulse:
    loc: str
    from_loc: str
    low: bool = True

    def flip(self):
        self.low = not self.low

In [5]:
from collections import deque

def part1(lines: list[str]) -> int:
    mappings = get_mappings(lines)

    conjunctions = []
    for k, v in mappings.items():
        if k == "broadcaster":
            continue
        (t, _, _) = v
        if t == "&":
            conjunctions.append(k)

    low_count = 0
    high_count = 0

    pulses = deque([])
    for _ in range(1000):

        # button press to broadcaster
        low_count += 1
        # Ok we're now ready to play
        for loc in mappings["broadcaster"]:
            low_count += 1
            pulses.append(Pulse(loc=loc, from_loc="broadcaster"))


        # get through current batch
        while pulses:
            pulse = pulses.popleft()
            assert isinstance(pulse, Pulse)

            if pulse.loc not in mappings:
                # at a dead end sort i think let it die
                continue
            
            type_, dest, state = mappings[pulse.loc]

            if type_ == "%":
                if pulse.low:
                    state = not state
                    mappings[pulse.loc] = (type_, dest, state)
                    if state:
                        # if now ON, sends a high pulse
                        pulse.low = False
                
                    for d in dest:
                        if pulse.low:
                            low_count += 1
                        else:
                            high_count += 1
                        pulses.append(
                            Pulse(loc=d, from_loc=pulse.loc, low=pulse.low)
                        )
                # if it recieves a high do nothing
            elif type_ == "&":
                # conjunction module
                assert isinstance(state, dict)
                state[pulse.from_loc] = not pulse.low
                if all(high for high in state.values()):
                    # all high values -> send low pulse
                    low=True
                else:
                    low=False
                for d in dest:
                    if low:
                        low_count += 1
                    else:
                        high_count += 1
                    pulses.append(
                        Pulse(loc=d, from_loc=pulse.loc, low=low)
                    )
            else:
                raise RuntimeError(f"what's going on: {type_=}")
            
    print(f"{low_count=} {high_count=}, {low_count*high_count=}")

    return low_count*high_count

assert part1(example1_lines) == 32000000
assert part1(example2_lines) == 11687500
part1(input_lines)

low_count=8000 high_count=4000, low_count*high_count=32000000
low_count=4250 high_count=2750, low_count*high_count=11687500
low_count=17422 high_count=48202, low_count*high_count=839775244


839775244

**Part 2- final machine responsible for moving sand is named `rx`. THe machine turns on when a single low pulse is sent to `rx`**

Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press... what is the fewest number of BUTTON PRESSES required to deliver a single *LOW PULSE* to the module named `rx`?

In [17]:
from collections import deque
import math

def part2(lines: list[str]) -> int:
    mappings = get_mappings(lines)

    conjunctions = []
    for k, v in mappings.items():
        if k == "broadcaster":
            continue
        (t, _, _) = v
        if t == "&":
            conjunctions.append(k)

    pulses = deque([])

    (feed,) = [name for name, mapping in mappings.items() if "rx" in mapping[1]]

    cycle_lengths = {}

    seen = {name: 0 for name, mapping in mappings.items() if feed in mapping[1]}

    button_presses = 0
    while True:
        button_presses += 1

        for loc in mappings["broadcaster"]:
            pulses.append(Pulse(loc=loc, from_loc="broadcaster"))

        while pulses:
            pulse = pulses.popleft()
            assert isinstance(pulse, Pulse)

            if pulse.loc not in mappings:
                continue
            
            type_, dest, state = mappings[pulse.loc]

            if pulse.loc == feed and not pulse.low:
                # this means we're going to feed into "rx" but with a high beam
                seen[pulse.from_loc] += 1

                if pulse.from_loc not in cycle_lengths:
                    # define cycle
                    cycle_lengths[pulse.from_loc] = button_presses
                else:
                    # ensure cycle assumption
                    assert button_presses == seen[pulse.from_loc] * cycle_lengths[pulse.from_loc]
                
            if all(seen.values()):
                return math.lcm(*cycle_lengths.values())


            if type_ == "%":
                if pulse.low:
                    state = not state
                    mappings[pulse.loc] = (type_, dest, state)
                    if state:
                        # if now ON, sends a high pulse
                        pulse.low = False
                
                    for d in dest:
                        if d == "rx" and pulse.low:
                            return button_presses

                        pulses.append(
                            Pulse(loc=d, from_loc=pulse.loc, low=pulse.low)
                        )

            else:
                # conjunction module
                assert isinstance(state, dict)
                state[pulse.from_loc] = not pulse.low
                if all(high for high in state.values()):
                    # all high values -> send low pulse
                    low=True
                else:
                    low=False
                for d in dest:
                    if d == "rx" and low:
                        return button_presses
                    pulses.append(
                        Pulse(loc=d, from_loc=pulse.loc, low=low)
                    )

part2(input_lines)

207787533680413

not a fan of part2